In [1]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keyboard
from abc import ABC, abstractmethod
import os
import csv

In [2]:
class Default_robot_interface(ABC):
    def __init__(self, paths = None, simulation = None):
        self.sim = simulation
        self.linear_velocity = 0
        self.rotational_velocity = 0
        self.sidewalk_velocity = 0
        
    @abstractmethod    
    def UPDATE_VEL(self):
        pass
        
    def Set_linear_velocity(self, value): 
        self.linear_velocity = value
        self.UPDATE_VEL()
        
    def Set_rotational_velocity(self, value):
        self.rotational_velocity = value
        self.UPDATE_VEL()
        
    def Set_sidewalk_velocity(self, value):
        self.sidewalk_velocity = value
        self.UPDATE_VEL()
        
    @abstractmethod     
    def Get_camera_image(self):
        pass
    @abstractmethod     
    def Get_camera_real_cords(self):
        pass

In [3]:
class Walker(Default_robot_interface):
    def __init__(self, paths = None, simulation = None):
        super().__init__(paths, simulation)
        
        self.Right_front_joint = paths[0]
        self.Right_rear_joint = paths[1]
        self.Left_front_joint = paths[2]
        self.Left_rear_joint = paths[3]
        self.Cam = paths[4]
        self.Dummy = paths[5]
        
    def UPDATE_VEL(self):
        
        self.sim.setJointTargetVelocity(self.Right_front_joint,
                                        self.linear_velocity - self.rotational_velocity + self.sidewalk_velocity)
        self.sim.setJointTargetVelocity(self.Right_rear_joint,
                                        self.linear_velocity - self.rotational_velocity - self.sidewalk_velocity)
        self.sim.setJointTargetVelocity(self.Left_front_joint,
                                        self.linear_velocity + self.rotational_velocity - self.sidewalk_velocity)
        self.sim.setJointTargetVelocity(self.Left_rear_joint,
                                        self.linear_velocity + self.rotational_velocity + self.sidewalk_velocity)
    def Get_camera_image(self):
        img, resX, resY = sim.getVisionSensorCharImage(self.Cam)
        img = np.frombuffer(img, dtype=np.uint8).reshape(resY, resX, 3)
        img = cv2.flip(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), 0)
        
        return img
        
    def Get_camera_real_cords(self, REL_POINT= None):
        if not REL_POINT:
            REL_POINT = self.sim.getObject("./Floor")
        pos = self.sim.getObjectPosition(self.Dummy, REL_POINT)
        
        Z_orient = self.sim.getObjectOrientation(self.Dummy, REL_POINT)[2]
        pos.append(Z_orient)
        return pos

In [4]:
class Simulation_loop_handler:
    def __init__(self, Default_robot_interface, LOOP_NODE_list, simulation):
        
        self.Robot = Default_robot_interface
        self.LOOP_NODE_list = LOOP_NODE_list
        self.loop_memory_dict = dict()
        self.sim = simulation
        self.running = False
        
    def start(self):
        
        self.sim.startSimulation()
        self.running = True
        while self.running:        
            for NODE in self.LOOP_NODE_list:
                if possible_returnment := NODE.call(self):
                    self.loop_memory_dict[possible_returnment[0]] = possible_returnment[1]
            self.sim.step()
            print(f'Simulation time: {self.sim.getSimulationTime():.2f} [s]',sep='', end='\r')
                          
        self.sim.stopSimulation()
        self.loop_memory_dict.clear()        

In [5]:
class NODE_interface(ABC):
    @abstractmethod
    def __init__(self, params = None):
        
        pass
        
    @abstractmethod
    def call(self, Simulation_loop_handler, values = None):
        
        pass

In [6]:
class SIM_TIME_LIMIT_CHECKER(NODE_interface):
    def __init__(self, params = None):
        
        self.time_limit = params[0]
        
    def call(self, Simulation_loop_handler):
        
        if Simulation_loop_handler.sim.getSimulationTime() >= self.time_limit:
            Simulation_loop_handler.running = False

In [7]:
class KEYBOARD_REMOTE_CONTROLLER(NODE_interface):
    def __init__(self, params = None):
        
        self.velocity_multiplier = params[0]
        
    def call(self, Simulation_loop_handler):
        
        if keyboard.is_pressed('w'):
            Simulation_loop_handler.Robot.Set_linear_velocity(self.velocity_multiplier)
        elif keyboard.is_pressed('s'):
            Simulation_loop_handler.Robot.Set_linear_velocity(-self.velocity_multiplier)
        else:
            Simulation_loop_handler.Robot.Set_linear_velocity(0)
            
        if keyboard.is_pressed('a'): 
            Simulation_loop_handler.Robot.Set_sidewalk_velocity(-self.velocity_multiplier)
        elif keyboard.is_pressed('d'):
            Simulation_loop_handler.Robot.Set_sidewalk_velocity(self.velocity_multiplier)
        else:
            Simulation_loop_handler.Robot.Set_sidewalk_velocity(0)
            
        if keyboard.is_pressed('q'):
            Simulation_loop_handler.Robot.Set_rotational_velocity(self.velocity_multiplier)
        elif keyboard.is_pressed('e'):
            Simulation_loop_handler.Robot.Set_rotational_velocity(-self.velocity_multiplier)
        else:
            Simulation_loop_handler.Robot.Set_rotational_velocity(0)
        

In [8]:
class IMAGE_LOADER(NODE_interface):
    def __init__(self, params = None):
        
        self.load_dir = params[0]

        if self.load_dir:
            for filename in os.listdir(self.load_dir):
                file_path = os.path.join(self.load_dir, filename)
                try:
                    if os.path.isfile(file_path):
                        os.remove(file_path)
                except Exception as e:
                    print(f'Ошибка при удалении файла {file_path}. {e}')
                    
    def call(self, Simulation_loop_handler):
        
        im = Simulation_loop_handler.Robot.Get_camera_image()
        Simulation_loop_handler.loop_memory_dict['last_gotten_image'] = im      
        if self.load_dir:
            cv2.imwrite(f"{self.load_dir}/{round(Simulation_loop_handler.sim.getSimulationTime(), 2)}.jpg", im)      

In [9]:
class TRUE_POS_GETTER(NODE_interface):
    def __init__(self, params = None):
        
        self.csv_path = params[0]
        self.relative_point = params[1]
        with open(self.csv_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([])
            
    def call(self, Simulation_loop_handler):
        
        pos = Simulation_loop_handler.Robot.Get_camera_real_cords(self.relative_point)
        
        Simulation_loop_handler.loop_memory_dict['current_position'] = pos
        
        with open(self.csv_path, 'a', newline='') as file:
            writer = csv.writer(file)
            pos.insert(0, round(Simulation_loop_handler.sim.getSimulationTime(), 2))
            writer.writerows([pos])

In [10]:
class IMAGE_SHOWER(NODE_interface):
    
    def __init__(self, params = None):
        pass
        
    def call(self, Simulation_loop_handler):
        
        plt.imshow(Simulation_loop_handler.loop_memory_dict['last_gotten_image'])        

In [11]:
class Motion_strategy_in_time:

    def __init__(self, time2command_list):
        
        self.time2command_list = time2command_list
        self.__current_instruction_number = 0

        if self.time2command_list[0][0]:
            self.time2command_list.insert(0, [0.0, [0, 0, 0]])

    def get_instruction(self, time):

        if (self.__current_instruction_number + 1 < len(self.time2command_list) and
            time >= self.time2command_list[self.__current_instruction_number + 1][0]):
            
            self.__current_instruction_number += 1

        return self.time2command_list[self.__current_instruction_number][1]

In [12]:
class PLAN_CONTROLLER(NODE_interface):
    
    def __init__(self, params = None):
        
        self.strategy = params[0]

    def call(self, Simulation_loop_handler):
        
        velocities = self.strategy.get_instruction(Simulation_loop_handler.sim.getSimulationTime())
        
        Simulation_loop_handler.Robot.Set_linear_velocity(velocities[0])
        Simulation_loop_handler.Robot.Set_sidewalk_velocity(velocities[1])
        Simulation_loop_handler.Robot.Set_rotational_velocity(velocities[2])

In [16]:
client = RemoteAPIClient()
sim = client.require('sim')
client.setStepping(True)



walker = Walker(paths = [sim.getObject("./Walker/Right_front_joint"),
                         sim.getObject("./Walker/Right_rear_joint"),
                         sim.getObject("./Walker/Left_front_joint"),
                         sim.getObject("./Walker/Left_rear_joint"),
                         sim.getObject("./Walker/Cam"),
                         sim.getObject("./Walker/Cam/dummy")],
                simulation = sim)

t_lim = SIM_TIME_LIMIT_CHECKER([25])
k_rem = KEYBOARD_REMOTE_CONTROLLER([5])
plan = Motion_strategy_in_time([[1,[5,0,0]],
                                [3, [0, 0, -5]],
                                [5, [4, 0, 0]],
                                [7, [0, 0, 4]],
                                [11, [5, 0, 0]],
                                [13, [0, 0, -5]],
                                [18,[5,0,0]]])

    
RC = PLAN_CONTROLLER([plan])
f_loader = IMAGE_LOADER(["./images_from_cam"])
#ish = IMAGE_SHOWER()
tpg = TRUE_POS_GETTER(['True_path.csv', sim.getObject("./START_POS")])
NODES = [t_lim,
         f_loader,
         k_rem,
         tpg]
MAIN_SIM = Simulation_loop_handler(Default_robot_interface = walker,
                                   LOOP_NODE_list = NODES,
                                   simulation = sim)
MAIN_SIM.start()

In [14]:
цййййййййййййййййййййййййййуууууууууууцццццццццццццццццццццццццццццццццууууууууууууууууууууууууууууууууууууууууууууууцццййццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццццййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййццццццйййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййцуййййййййййййййййййййййййййййййййййййййййййййййцууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууййййййййййййййййййййййййййййййййййййййййwwwwwwwwwaaaaaaaaaaaaaaaqqqqqwwwwwwwwwwwwwwwwwwwwwwwwwqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwweeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeewwwwewwwwweweeeeeeeeqqqqqqqqqqqqqqqqqqqwqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqйййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййййwwwwwwwwwwwwwwwwweeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqwwwwwwwwwwwwwwwwwwwwwwwqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqwwwwwwwwwweqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqwwwwwwwwwwwwwwwwwwwww

NameError: name 'цййййййййййййййййййййййййййууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууууу' is not defined